# Purview Metadata Uploader

Este notebook actualiza metadatos de Business Metadata en Microsoft Purview para assets de Fabric Lakehouse.

## Prerequisitos
- Service Principal con permisos en Purview (Data Curator)
- Tabla `tablas_metadata` en el Lakehouse
- Business Metadata "DataGovernance" creado en Purview

## Configuración
Actualiza las variables en la celda siguiente con tus credenciales.

In [ ]:
# =============================================================================
# CONFIGURACIÓN
# =============================================================================

# Purview Configuration
PURVIEW_ACCOUNT_NAME = "YOUR_PURVIEW_ACCOUNT_NAME"  # Nombre de tu cuenta de Purview
PURVIEW_ENDPOINT = f"https://{PURVIEW_ACCOUNT_NAME}.purview.azure.com"

# Service Principal Authentication
TENANT_ID = "XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX"  # Azure AD Tenant ID
CLIENT_ID = "XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX"  # Service Principal Client ID
CLIENT_SECRET = "YOUR_CLIENT_SECRET"  # Service Principal Secret

# Metadata Configuration
BUSINESS_METADATA_NAME = "DataGovernance"  # Nombre del Business Metadata en Purview
DEFAULT_OWNER_EMAIL = "admin@yourcompany.com"  # Email por defecto

# Lakehouse Tables
TABLES_METADATA_TABLE = "tablas_metadata"  # Tabla con metadata de tablas
COLUMNS_METADATA_TABLE = "campos_metadata"  # Tabla con metadata de columnas (opcional)

print("✅ Configuración cargada")
print(f"Purview: {PURVIEW_ACCOUNT_NAME}")
print(f"Business Metadata: {BUSINESS_METADATA_NAME}")

In [ ]:
# =============================================================================
# INSTALAR DEPENDENCIAS
# =============================================================================

import sys
import subprocess

print("Instalando dependencias...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", 
                      "azure-purview-catalog", "azure-identity", "requests"])

print("✅ Dependencias instaladas")

In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================

import pandas as pd
import requests
import json
from azure.identity import ClientSecretCredential
from azure.purview.catalog import PurviewCatalogClient
from pyspark.sql import SparkSession

print("✅ Imports completados")

In [ ]:
# =============================================================================
# LEER DATOS DEL LAKEHOUSE
# =============================================================================

print("Leyendo datos del Lakehouse...")

# Leer tabla de metadata
spark_df_tablas = spark.read.table(TABLES_METADATA_TABLE)
df_tablas = spark_df_tablas.toPandas()

print(f"✅ Datos leídos: {len(df_tablas)} tablas")
print(f"\nColumnas disponibles: {list(df_tablas.columns)}")
print(f"\nPrimeras 3 filas:")
display(df_tablas.head(3))

In [ ]:
# =============================================================================
# AUTENTICACIÓN
# =============================================================================

print("Autenticando con Purview...")

# Crear credential
credential = ClientSecretCredential(
    tenant_id=TENANT_ID,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

# Crear cliente de Purview
client = PurviewCatalogClient(
    endpoint=PURVIEW_ENDPOINT,
    credential=credential
)

# Test de conexión
try:
    test = client.discovery.query(search_request={"keywords": "*", "limit": 1})
    print(f"✅ Autenticado correctamente")
    print(f"   Assets encontrados en Purview: {len(test.get('value', []))}")
except Exception as e:
    print(f"❌ Error de autenticación: {str(e)}")
    raise

In [ ]:
# =============================================================================
# FUNCIONES AUXILIARES
# =============================================================================

def safe_str(value):
    """Convierte valor a string, manejando NaN/None"""
    if pd.isna(value):
        return None
    return str(value).strip()

def search_table_in_purview(client, table_name):
    """Busca una tabla en Purview por nombre"""
    try:
        search_request = {
            "keywords": table_name,
            "filter": {"entityType": "fabric_lakehouse_table"},
            "limit": 10
        }
        response = client.discovery.query(search_request=search_request)
        
        if not response or 'value' not in response or len(response['value']) == 0:
            return None
        
        for asset in response['value']:
            if asset.get('name', '').lower() == table_name.lower():
                return {
                    'guid': asset.get('id'),
                    'qualifiedName': asset.get('qualifiedName'),
                    'typeName': asset.get('entityType'),
                    'name': asset.get('name')
                }
        return None
    except:
        return None

print("✅ Funciones auxiliares definidas")

In [ ]:
# =============================================================================
# CREAR BUSINESS METADATA DEFINITION
# =============================================================================

def create_business_metadata_def(business_metadata_name):
    """Crea Business Metadata definition en Purview"""
    try:
        token = credential.get_token('https://purview.azure.net/.default').token
        url = f"{PURVIEW_ENDPOINT}/catalog/api/atlas/v2/types/typedefs"
        headers = {
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        }
        
        payload = {
            "businessMetadataDefs": [{
                "category": "BUSINESS_METADATA",
                "name": business_metadata_name,
                "description": f"{business_metadata_name} metadata",
                "typeVersion": "1.0",
                "attributeDefs": [
                    {
                        "name": "Expert",
                        "typeName": "string",
                        "isOptional": True,
                        "cardinality": "SINGLE",
                        "options": {
                            "maxStrLength": "500",
                            "applicableEntityTypes": '["fabric_lakehouse_table"]'
                        }
                    },
                    {
                        "name": "Owner",
                        "typeName": "string",
                        "isOptional": True,
                        "cardinality": "SINGLE",
                        "options": {
                            "maxStrLength": "500",
                            "applicableEntityTypes": '["fabric_lakehouse_table"]'
                        }
                    },
                    {
                        "name": "Source",
                        "typeName": "string",
                        "isOptional": True,
                        "cardinality": "SINGLE",
                        "options": {
                            "maxStrLength": "200",
                            "applicableEntityTypes": '["fabric_lakehouse_table"]'
                        }
                    },
                    {
                        "name": "Refresh_Frequency",
                        "typeName": "string",
                        "isOptional": True,
                        "cardinality": "SINGLE",
                        "options": {
                            "maxStrLength": "100",
                            "applicableEntityTypes": '["fabric_lakehouse_table"]'
                        }
                    },
                    {
                        "name": "Data_Sensitivity",
                        "typeName": "string",
                        "isOptional": True,
                        "cardinality": "SINGLE",
                        "options": {
                            "maxStrLength": "50",
                            "applicableEntityTypes": '["fabric_lakehouse_table"]'
                        }
                    },
                    {
                        "name": "Retention_Period",
                        "typeName": "string",
                        "isOptional": True,
                        "cardinality": "SINGLE",
                        "options": {
                            "maxStrLength": "100",
                            "applicableEntityTypes": '["fabric_lakehouse_table"]'
                        }
                    },
                    {
                        "name": "Business_Owner",
                        "typeName": "string",
                        "isOptional": True,
                        "cardinality": "SINGLE",
                        "options": {
                            "maxStrLength": "200",
                            "applicableEntityTypes": '["fabric_lakehouse_table"]'
                        }
                    }
                ]
            }]
        }
        
        response = requests.post(url, json=payload, headers=headers, timeout=10)
        
        if response.status_code in [200, 201]:
            print(f"✅ Business Metadata '{business_metadata_name}' creado")
            return True
        elif response.status_code == 409:
            print(f"ℹ️  Business Metadata '{business_metadata_name}' ya existe")
            return True
        else:
            print(f"❌ Error {response.status_code}: {response.text[:300]}")
            return False
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False

# Crear o verificar Business Metadata
print(f"\nCreando Business Metadata '{BUSINESS_METADATA_NAME}'...")
create_business_metadata_def(BUSINESS_METADATA_NAME)

In [ ]:
# =============================================================================
# TEST CON 1 TABLA
# =============================================================================

print("="*70)
print("TEST: Procesando primera tabla")
print("="*70)

# Tomar primera tabla
first_table = df_tablas.iloc[0]
table_name = safe_str(first_table['Table'])

print(f"\nTabla: {table_name}")

# Buscar en Purview
asset_info = search_table_in_purview(client, table_name)

if not asset_info:
    print(f"❌ Tabla no encontrada en Purview")
else:
    guid = asset_info['guid']
    print(f"✅ Asset encontrado: {guid[:20]}...")
    
    # Preparar business metadata
    business_metadata = {
        BUSINESS_METADATA_NAME: {
            "Expert": safe_str(first_table.get('Expert')) or DEFAULT_OWNER_EMAIL,
            "Owner": safe_str(first_table.get('Owner')) or DEFAULT_OWNER_EMAIL,
            "Source": safe_str(first_table.get('Source')) or '',
            "Refresh_Frequency": safe_str(first_table.get('Refresh_Frequency')) or '',
            "Data_Sensitivity": safe_str(first_table.get('Data_Sensitivity')) or '',
            "Retention_Period": safe_str(first_table.get('Retention_Period')) or '',
            "Business_Owner": safe_str(first_table.get('Business_Owner')) or ''
        }
    }
    
    print(f"\nMetadata a actualizar:")
    for key, value in business_metadata[BUSINESS_METADATA_NAME].items():
        if value:
            print(f"  - {key}: {value}")
    
    # Asignar metadata
    token = credential.get_token('https://purview.azure.net/.default').token
    url = f"{PURVIEW_ENDPOINT}/catalog/api/atlas/v2/entity/guid/{guid}/businessmetadata"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    response = requests.post(url, json=business_metadata, headers=headers, timeout=30)
    
    if response.status_code in [200, 201, 204]:
        print(f"\n✅ SUCCESS!")
        print(f"\nVerifica en Purview:")
        print(f"  https://web.purview.azure.com")
        print(f"  Busca: {table_name}")
        print(f"  Overview > Data asset attributes > {BUSINESS_METADATA_NAME}")
    else:
        print(f"\n❌ Error {response.status_code}")
        print(f"   {response.text[:500]}")

In [ ]:
# =============================================================================
# PROCESAR TODAS LAS TABLAS
# =============================================================================

print("="*70)
print(f"PROCESANDO {len(df_tablas)} TABLAS")
print("="*70)

success_count = 0
error_count = 0
not_found_count = 0
errors = []

for idx, table_row in df_tablas.iterrows():
    table_name = safe_str(table_row['Table'])
    
    if not table_name:
        continue
    
    print(f"\n[{idx+1}/{len(df_tablas)}] {table_name}")
    
    try:
        # Buscar asset
        asset_info = search_table_in_purview(client, table_name)
        
        if not asset_info:
            print(f"  ⏭️  No encontrada en Purview")
            not_found_count += 1
            continue
        
        guid = asset_info['guid']
        
        # Preparar business metadata
        business_metadata = {
            BUSINESS_METADATA_NAME: {
                "Expert": safe_str(table_row.get('Expert')) or DEFAULT_OWNER_EMAIL,
                "Owner": safe_str(table_row.get('Owner')) or DEFAULT_OWNER_EMAIL,
                "Source": safe_str(table_row.get('Source')) or '',
                "Refresh_Frequency": safe_str(table_row.get('Refresh_Frequency')) or '',
                "Data_Sensitivity": safe_str(table_row.get('Data_Sensitivity')) or '',
                "Retention_Period": safe_str(table_row.get('Retention_Period')) or '',
                "Business_Owner": safe_str(table_row.get('Business_Owner')) or ''
            }
        }
        
        # Asignar metadata
        token = credential.get_token('https://purview.azure.net/.default').token
        url = f"{PURVIEW_ENDPOINT}/catalog/api/atlas/v2/entity/guid/{guid}/businessmetadata"
        headers = {
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        }
        
        response = requests.post(url, json=business_metadata, headers=headers, timeout=30)
        
        if response.status_code in [200, 201, 204]:
            print(f"  ✅ OK")
            success_count += 1
        else:
            print(f"  ❌ Error {response.status_code}")
            error_count += 1
            errors.append(f"{table_name}: HTTP {response.status_code}")
            
    except Exception as e:
        print(f"  ❌ Exception: {str(e)}")
        error_count += 1
        errors.append(f"{table_name}: {str(e)}")

# Resumen
print("\n" + "="*70)
print("RESUMEN FINAL")
print("="*70)
print(f"Total tablas: {len(df_tablas)}")
print(f"Actualizadas: {success_count}")
print(f"Errores: {error_count}")
print(f"No encontradas: {not_found_count}")

if errors:
    print(f"\nErrores:")
    for error in errors:
        print(f"  - {error}")

if success_count > 0:
    print("\n" + "="*70)
    print("✅ PROCESO COMPLETADO!")
    print("="*70)
    print(f"\n{success_count} tablas actualizadas con Business Metadata")
    print(f"\nVerifica en Purview Portal:")
    print(f"  https://web.purview.azure.com")